In [16]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [27]:
dataset_path = "/home/yadagiri/train_dataset.csv"
df = pd.read_csv(dataset_path)

In [28]:
df

,id,Category,text,Vocabulary,Noun Count,Verb Count,AUX Count,NUM Count,PRON Count,ADV Count,INTJ Count,PART Count,label
0,text-bison-002/news-2021-01-01-2021-12-31-capi...,text-bison,Unveiling the Plot: Evidence Points to Premedi...,279,146,86,2,2,9,13,0,0,1
1,gpt-3.5-turbo-0125/news-2021-01-01-2021-12-31-...,gpt-3.5-turbo,Chris Wallace Departs Fox News After 18 Years ...,215,85,48,1,5,12,5,0,1,1
2,mistralai-mixtral-8x7b-instruct-v0.1/news-2021...,mistralai-mixtral-8x7b,Tiger Woods' Resurgence: A Journey from Scanda...,255,107,64,4,10,31,13,0,0,1
3,text-bison-002/news-2021-01-01-2021-12-31-bide...,text-bison,Fact Check: Biden Inauguration Amidst Pandemic...,282,130,84,0,6,7,9,0,1,1
4,bigscience-bloomz-7b1/news-2021-01-01-2021-12-...,bigscience-bloomz,"Hurricane Ida, the third major hurricane to hi...",114,66,58,2,11,4,4,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12145,mistralai-mixtral-8x7b-instruct-v0.1/news-2021...,mistralai-mixtral-8x7b,"Tom Brady Retires, Baker Mayfield Takes Over: ...",261,121,64,10,25,9,10,0,1,1
12146,bigscience-bloomz-7b1/news-2021-01-01-2021-12-...,bigscience-bloomz,"NEW YORK (CNN) -- On the morning of Sept. 11, ...",125,37,36,0,4,17,7,0,1,1
12147,vicgalle-gpt2-open-instruct-v1/news-2021-01-01...,vicgalle-gpt2,Tiger Woods was involved in a serious car cras...,147,103,86,1,1,32,16,0,1,1
12148,articles-cleaned-truncated/news-2021-01-01-202...,Human,Facebook warns of slower revenue growth amid f...,233,106,54,1,17,10,10,0,2,0


In [22]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Load dataset
dataset_path = "/home/yadagiri/train_dataset.csv"
df = pd.read_csv(dataset_path)

X = df['text'].astype(str).tolist()
#y = np.asarray(d1[d1.columns[1:]])
# Convert text data to numerical features
vectorizer = CountVectorizer()
text_data_numeric = vectorizer.fit_transform(X).toarray()
numerical_data = df[['Vocabulary', 'Noun Count', 'Verb Count', 'AUX Count', 'NUM Count', 'PRON Count', 'ADV Count', 'INTJ Count', 'PART Count']].values
# Convert text data to a NumPy array
# text_data = np.array(X)
# Convert text data to a NumPy array with a single column
#text_data = np.array(X).reshape(-1, 1)

# Concatenate text and numerical data
X = np.concatenate((text_data_numeric, numerical_data), axis=1)
y_data = df[df.columns[12:13]]
y = y_data.values
######### Label Encoding for y_train & y_test################
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y
# Split dataset into train and validation sets
train_X, val_X, train_y, val_y = train_test_split(X,y, test_size=0.1, random_state=42)

class CustomDataset(Dataset):
  def __init__(self, X, y, tokenizer):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    # Convert numerical data (X[idx]) to text representation (text)
    text = self.convert_to_text(X[idx])  # Replace with your conversion logic

    # Tokenize the text
    encoding = self.tokenizer(text, return_tensors="pt")

    return {
        "input_ids": encoding["input_ids"].squeeze(),
        "attention_mask": encoding["attention_mask"].squeeze(),
        "labels": torch.tensor(self.y[idx])
    }

# Define a separate function to convert numerical data to text (if applicable)
def convert_to_text(numerical_data):
  
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Assuming 2 classes for binary classification
batch_size = 32
epochs = 3
# Create DataLoader for train and validation sets
train_dataset = CustomDataset(train_X, train_y, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CustomDataset(val_X, val_y, tokenizer)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define optimizer and loss function
# optimizer = AdamW(model.parameters(), lr=learning_rate)
# criterion = torch.nn.CrossEntropyLoss()

# Create DataLoader for train and validation sets
# train_dataset = CustomDataset(train_df['text'], train_df['binary_label'], tokenizer)
# val_dataset = CustomDataset(val_df['text'], val_df['binary_label'], tokenizer)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define training parameters

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = AdamW(model.parameters(), lr=2e-5)



SyntaxError: incomplete input (4104458842.py, line 80)

In [23]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Load dataset
dataset_path = "/home/yadagiri/train_dataset.csv"
df = pd.read_csv(dataset_path)

X = df['text'].astype(str).tolist()
y_data = df[df.columns[12:13]]  # Assuming the label column is at index 12
y = y_data.values.ravel()  # Flatten the label array

# Label Encoding (if necessary)
# If your labels are not already numerical, uncomment these lines:
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(y)

# Split dataset into train and validation sets
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.1, random_state=42)

class CustomDataset(Dataset):
  def __init__(self, X, y, tokenizer):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    # Convert numerical data (X[idx]) to text representation (text)
    text = self.convert_to_text(X[idx])  # Replace with your conversion logic (if applicable)

    # Tokenize the text
    encoding = self.tokenizer(text, return_tensors="pt")

    return {
        "input_ids": encoding["input_ids"].squeeze(),
        "attention_mask": encoding["attention_mask"].squeeze(),
        "labels": torch.tensor(self.y[idx])
    }

# Define a separate function to convert numerical data to text (if applicable)
def convert_to_text(numerical_data):
  # Implement your logic here based on the chosen approach (consider feature engineering if needed)
  # This function should return the text representation of the numerical data (if applicable)
  ...

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(np.unique(y)))  # Set num_labels based on unique classes

batch_size = 32
epochs = 3

# Create DataLoaders for train and validation sets
train_dataset = CustomDataset(train_X, train_y, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CustomDataset(val_X, val_y, tokenizer)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define optimizer and loss function (consider using EarlyStopping for better training)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()  # Assuming binary classification

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [29]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Load dataset
dataset_path = "/home/yadagiri/train_dataset.csv"
df = pd.read_csv(dataset_path)

# Text data to features (if needed, consider alternative feature extraction techniques)
# You can comment out this section if you don't want to use TF-IDF
# vectorizer = CountVectorizer()
# text_data_numeric = vectorizer.fit_transform(df['text']).toarray()

# Concatenate text features (if used) with numerical data
numerical_data = df[['Vocabulary', 'Noun Count', 'Verb Count', 'AUX Count', 'NUM Count', 'PRON Count', 'ADV Count', 'INTJ Count', 'PART Count']].values
X = df['text'].tolist()  # Assuming text data is strings

# Concatenation logic (choose appropriate strategy based on feature types)
combined_data = []
for text, features in zip(X, numerical_data):
  # Option 1: Direct Concatenation (if features directly relate to text)
  # combined_text = f"{text}, {', '.join([str(f) for f in features])}"

  # Option 2: Categorical Encoding and Concatenation (if features represent categories)
  # You'll need to implement categorical encoding logic here (e.g., using LabelEncoder)
  # combined_text = ... (your logic for categorical encoding and concatenation)

  # Option 3: Custom Textual Descriptions (for more complex relationships)
  # You'll need to implement logic to create textual descriptions from features here
  # combined_text = ... (your logic for creating textual descriptions and concatenation)

  combined_data.append(combined_text)

# Convert combined data to NumPy array
X = np.array(combined_data)

y_data = df[df.columns[12:13]]
y = y_data.values

# Label Encoding for y_train & y_test
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y

# Split dataset into train and validation sets
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.1, random_state=42)

class CustomDataset(Dataset):
  def __init__(self, X, y, tokenizer):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    text = self.X[idx]

    # Tokenize the text
    encoding = self.tokenizer(text, return_tensors="pt")

    return {
        "input_ids": encoding["input_ids"].squeeze(),
        "attention_mask": encoding["attention_mask"].squeeze(),
        "labels": torch.tensor(self.y[idx])
    }

# Define a separate function to convert numerical data to text (if applicable)
def convert_to_text(numerical_data):
  # Implement your logic here based on the chosen approach (consider feature engineering if needed)
  # This function should return the text representation of the numerical data (if applicable)
  ...

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Assuming 2 classes for binary classification
batch_size = 32
epochs = 3

# Create DataLoaders for train and validation sets
train_dataset = CustomDataset(train_X, train_y, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CustomDataset(val_X, val_y, tokenizer)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

NameError: name 'combined_text' is not defined

In [24]:
from sklearn.metrics import precision_score, recall_score, f1_score
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Initialize empty lists to store loss and accuracy values
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
train_precisions = []
val_precisions = []
train_recalls = []
val_recalls = []
train_f1_scores = []
val_f1_scores = []

for epoch in range(epochs):
    model.train()
    epoch_train_loss = 0
    correct_train_preds = 0
    total_train_preds = 0
    train_predicted_labels = []
    train_true_labels = []
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        epoch_train_loss += loss.item()
        _, predicted = torch.max(outputs.logits, 1)
        correct_train_preds += (predicted == labels).sum().item()
        total_train_preds += labels.size(0)
        train_predicted_labels.extend(predicted.cpu().numpy())
        train_true_labels.extend(labels.cpu().numpy())

    # Calculate training accuracy and loss for the epoch
    train_loss = epoch_train_loss / len(train_loader)
    train_accuracy = correct_train_preds / total_train_preds
    train_precision = precision_score(train_true_labels, train_predicted_labels, average='weighted')
    train_recall = recall_score(train_true_labels, train_predicted_labels, average='weighted')
    train_f1 = f1_score(train_true_labels, train_predicted_labels, average='weighted')

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    train_precisions.append(train_precision)
    train_recalls.append(train_recall)
    train_f1_scores.append(train_f1)

    # Validation loop
    model.eval()
    epoch_val_loss = 0
    correct_val_preds = 0
    total_val_preds = 0
    val_predicted_labels = []
    val_true_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            epoch_val_loss += loss.item()
            _, predicted = torch.max(outputs.logits, 1)
            correct_val_preds += (predicted == labels).sum().item()
            total_val_preds += labels.size(0)
            val_predicted_labels.extend(predicted.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())

    # Calculate validation accuracy and loss for the epoch
    val_loss = epoch_val_loss / len(val_loader)
    val_accuracy = correct_val_preds / total_val_preds
    val_precision = precision_score(val_true_labels, val_predicted_labels, average='weighted')
    val_recall = recall_score(val_true_labels, val_predicted_labels, average='weighted')
    val_f1 = f1_score(val_true_labels, val_predicted_labels, average='weighted')

    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    val_precisions.append(val_precision)
    val_recalls.append(val_recall)
    val_f1_scores.append(val_f1)

    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss}, Train Accuracy: {train_accuracy}, Train Precision: {train_precision}, Train Recall: {train_recall}, Train F1: {train_f1}")
    print(f"Val Loss: {val_loss}, Val Accuracy: {val_accuracy}, Val Precision: {val_precision}, Val Recall: {val_recall}, Val F1: {val_f1}")


AttributeError: 'CustomDataset' object has no attribute 'convert_to_text'

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
# Load test dataset
test_dataset_path = ""
test_df = pd.read_csv(test_dataset_path)
X = list(test_df['text'])
# #y = np.asarray(d1[d1.columns[1:]])
y_data = test_df[test_df.columns[12:13]]
y = y_data.values
######### Label Encoding for y_train & y_test################
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
test_df['label'] = label_encoder.fit_transform(test_df['label'])

# Create DataLoader for test set
test_dataset = CustomDataset(test_df['text'], test_df['label'], tokenizer)  # Assuming 'binary_label' is the name of the label column
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# Initialize empty lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Evaluate model on test set
model.eval()
test_loss = 0
correct_preds = 0
total_preds = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        test_loss += loss.item()
        _, predicted = torch.max(logits, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Calculate average test loss and accuracy
avg_test_loss = test_loss / len(test_loader)
test_accuracy = correct_preds / total_preds

# Calculate precision, recall, and F1 score
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# Print metrics
print(f"Test Loss: {avg_test_loss}, Test Accuracy: {test_accuracy}")
print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)
